# 201 MAGICC7 input scenarios

In [1]:
ENSEMBLE_MEMBERS=5

In [2]:
import json
import os
import platform
from pathlib import Path

import dotenv
import matplotlib.pyplot as plt
import openscm_runner
import scmdata
import pyam

from climate_assessment.climate.wg3 import clean_wg3_scenarios
from climate_assessment.climate.magicc7 import get_magicc7_configurations

from utils import rebase_temperatures_wg3

%load_ext nb_black

<IPython.core.display.Javascript object>

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/scmdata/database/_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


<IPython.core.display.Javascript object>

In [3]:
dotenv.load_dotenv()

True

<IPython.core.display.Javascript object>

In [4]:
os_type = platform.system()
print(os_type)

Darwin


<IPython.core.display.Javascript object>

In [5]:
match os_type:
    case 'Darwin':
        os.environ['MAGICC_EXECUTABLE_7']=os.path.join(
            os.environ['MAGICC_7_BIN'],
            'magicc-darwin-amd64'
        )
        os.environ['DYLD_LIBRARY_PATH'] = os.environ['DYLD_PATH_GCC']
    case 'Linux':
        os.environ['MAGICC_EXECUTABLE_7']=os.path.join(
            os.environ['MAGICC_7_BIN'],
            'magicc'
        )

<IPython.core.display.Javascript object>

## Read in the necessary data
We want to read in the original scenarios to run them through two MAGICC modes:
1. All greenhouse gas emissions
2. CO2 driven run

Step 1: Read in the emission data

In [6]:
df = pyam.IamDataFrame(
    os.path.join(
        '..',
        'data',
        '100_scenarios.csv'
    )
)

pyam.core - INFO: Reading file ../data/100_scenarios.csv


<IPython.core.display.Javascript object>

Step 2: We want to read in the MAGICC7 configuration for all GHG and CO2 only.

In [7]:
all_magicc_cfgs,_ = get_magicc7_configurations(
    magicc_version='v7.5.3',
    magicc_probabilistic_file=os.environ['MAGICC_AR6_PROBABILISTIC_DISTRIBUTION'],
    magicc_extra_config=None,
    num_cfgs=ENSEMBLE_MEMBERS,
    co2_and_non_co2_warming=True
)

<IPython.core.display.Javascript object>

Step 3: Pull out the configs for the GHG run

In [8]:
magicc_all_cfg = [x for x in all_magicc_cfgs if x['rf_total_runmodus']=='ALL']

<IPython.core.display.Javascript object>

Step 4: Pull out the configs for the CO2 run

In [9]:
magicc_co2_cfg = [x for x in all_magicc_cfgs if x['rf_total_runmodus']=='CO2']

<IPython.core.display.Javascript object>

In [10]:
assert len(magicc_all_cfg) == ENSEMBLE_MEMBERS

<IPython.core.display.Javascript object>

In [11]:
assert len(magicc_co2_cfg) == ENSEMBLE_MEMBERS

<IPython.core.display.Javascript object>

## Running MAGICC
Step 1: Clean the input scenarios.

In [12]:
input_scm = scmdata.ScmRun(
    clean_wg3_scenarios(
        df
    )
)

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/climate_assessment/climate/wg3.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean["variable"] = df_clean["variable"].str.replace(old, new)


<IPython.core.display.Javascript object>

Step 2: Run MAGICC for all GHG

In [13]:
temp_all_ghg = openscm_runner.run(
    {'MAGICC7':magicc_all_cfg},
    input_scm,
    output_variables=[
        'Surface Temperature'
    ]
)

Climate models:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

Historical data has not been checked


/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/openscm_units/_unit_registry.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for label, val in metric_conversion.iteritems():


Writing SCEN7 files:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

Front serial:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

...

Front parallel:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUB

Parallel runs:   0%|          | 0.00/6.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUB

<IPython.core.display.Javascript object>

Step 3: Run MAGICC7 driven by CO2 only

In [14]:
temp_co2 = openscm_runner.run(
    {'MAGICC7':magicc_co2_cfg},
    input_scm,
    output_variables=[
        'Surface Temperature'
    ]
)

Climate models:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

Historical data has not been checked


Writing SCEN7 files:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

Front serial:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUB

Front parallel:   0%|          | 0.00/2.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUB

Parallel runs:   0%|          | 0.00/6.00 [00:00<?, ?it/s]

/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATUPTK_EBALANCE_SUBANN.BINOUT. Monthly data are not read in automatically by `run`. Use `MAGICCData` instead.
  warnings.warn(
/Users/gauravganti/opt/anaconda3/envs/cdr_climate_uncertainty/lib/python3.10/site-packages/pymagicc/core.py:387: UserWarning: Not reading file: DAT_HEATCONT_EBALANCE_SUB

<IPython.core.display.Javascript object>

Step 4: Filter the "World" region in both dataframes.

In [15]:
temp_all_ghg_world = temp_all_ghg.filter(
    region='World'
)

<IPython.core.display.Javascript object>

In [16]:
temp_co2_world = temp_co2.filter(
    region='World'
)

<IPython.core.display.Javascript object>

Step 5: Now, we want to calculate the temperature rise relative to the 1850-1900 average.

In [17]:
temp_all_ghg_world_1850_1900 = (
    rebase_temperatures_wg3(
        temp_all_ghg_world
    )
)

<IPython.core.display.Javascript object>

In [18]:
temp_co2_world_1850_1900 = (
    rebase_temperatures_wg3(
        temp_co2_world
    )
)

<IPython.core.display.Javascript object>

## Write out the data

In [19]:
temp_all_ghg_world_1850_1900.to_csv(
    Path(
        '../data/201_original_all_ghg_temps.csv'
    )
)

<IPython.core.display.Javascript object>

In [20]:
temp_co2_world_1850_1900.to_csv(
    Path(
        '../data/201_original_co2_only_temps.csv'
    )
)

<IPython.core.display.Javascript object>